In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os
import soundfile as sf
import math
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


In [2]:
with tf.compat.v1.Session() as sess:
  devices = sess.list_devices()
devices

os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.compat.v1.ConfigProto( )
config.gpu_options.allow_growth = True
sess   =  tf.compat.v1.Session(config=config)

import tensorflow.keras.backend as tf_bkend
tf.compat.v1.keras.backend.set_session(sess)

### Define Path to Segmented Files

In [3]:
# Path to destination files 3-fold, 22050 * 1s = 110,250 inputs, overlapping 75 
path3      = '../GTzan_3f_Win_110250_75_safaa'
frame_size = 110250
nb_classes = 10

### List Files

In [4]:
file_list = []
i = 0 
for path, subdirs, files in os.walk( path3 ):
    for name in files:
        file_list.append( os.path.join( path, name) )
        i += 1
print("Files processed: "+str(i) )

Files processed: 20990


### Define Test Fold

In [5]:
Test  = "fold1"

### Put all Filenames in Two Lists

In [6]:
test_list  = []
test_path  = []
test_label_list  = []
test_seg_list  = []


choices = {'blues': 0, 
           'classical': 1, 
           'country': 2, 
           'disco': 3, 
           'hiphop': 4, 
           'jazz': 5,
           'metal': 6, 
           'pop': 7, 
           'reggae': 8, 
           'rock': 9}


i = 0 
for path, subdirs, files in os.walk( path3 ):
    for name in files:
        fold_id    = path.split('/')[2]
        # Replace the 2-character string by a number according to choices
        label_id     = choices.get( path.split('/')[3], 99 )
        track_id     = name[0:].split('.')[0]
        seg_id       = int(name.split('.')[0].split('_')[1])
        full_path_id = path+'/'+name
        if fold_id == Test:
            test_list.append( track_id )
            test_path.append( full_path_id )
            test_label_list.append( label_id )
            test_seg_list.append( seg_id )
        i += 1
print("Files processed: "+str(i) )
print("Test  files    : "+str(len(test_list)) )

Files processed: 20990
Test  files    : 7011


## Define CNN Model

In [7]:
def model_generator_alenet_nov19_a():
    from keras.layers import Input, Dense
    from keras.layers import AveragePooling1D, Conv1D, MaxPool1D, Flatten
    from keras.models import Model
    from keras import initializers
    from keras.callbacks import ModelCheckpoint
    from keras.layers import LeakyReLU    
    
    from keras import optimizers
    from keras.layers.normalization import BatchNormalization
    from keras.layers.core import Dropout
    
    import keras.initializers as init
    from keras.regularizers import l2
    
    inp =  Input( shape = (frame_size,1) )
    #----------------------
    conv1 = Conv1D(filters = 128, kernel_size = 3, strides = 3, activation ='relu', kernel_initializer = 'he_uniform', padding = 'valid', trainable = False)(inp)
    #norm1 = BatchNormalization()(conv1)
    #----------------------
    conv2 = Conv1D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(conv1)
    #norm2 = BatchNormalization()(conv2)
    pool2 = MaxPool1D(pool_size = 3, strides = 3)(conv2)
    #----------------------
    conv3 = Conv1D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool2)
    #norm3 = BatchNormalization()(conv3)
    pool3 = MaxPool1D(pool_size = 3, strides = 3)(conv3)
    #----------------------
    conv4 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool3)
    #norm4 = BatchNormalization()(conv4)
    pool4 = MaxPool1D(pool_size = 3, strides = 3)(conv4)
    #----------------------
    conv5 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool4)
    #norm5 = BatchNormalization()(conv5)
    pool5 = MaxPool1D(pool_size = 3, strides = 3)(conv5)
    #----------------------
    conv6 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool5)
    #norm6 = BatchNormalization()(conv6)
    pool6 = MaxPool1D(pool_size = 3, strides = 3)(conv6)
    #----------------------
    conv7 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool6)
    #norm7 = BatchNormalization()(conv7)
    pool7 = MaxPool1D(pool_size = 3, strides = 3)(conv7)
    #----------------------
    conv8 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool7)
    #norm8 = BatchNormalization()(conv8)
    pool8 = MaxPool1D(pool_size = 3, strides = 3)(conv8)
    #----------------------
    conv9 = Conv1D(filters = 256, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool8)
    #norm9 = BatchNormalization()(conv9)
    pool9 = MaxPool1D(pool_size = 3, strides = 3)(conv9)
    #----------------------
    conv10 = Conv1D(filters = 512, kernel_size = 3, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same' )(pool9)
    #norm10 = BatchNormalization()(conv10)
    pool10 = MaxPool1D(pool_size = 3, strides = 3)(conv10)
    #----------------------
    conv11 = Conv1D(filters = 512, kernel_size = 1, strides = 1, activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same')(pool10)
    #norm11 = BatchNormalization()(conv11)
    drop1  = Dropout(0.5)(conv11)
    
    flat = Flatten()(drop1)
    
    dense1 = Dense(nb_classes, activation='sigmoid')(flat)
    model  = Model(inp, dense1)

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
              ,metrics=['accuracy'])
    
    model.summary()
    
    return model

## Load learned weights

In [8]:
# Re-generating the model 
# model = model_generator_alenet4()
model = model_generator_alenet_nov19_a()

model.load_weights("XXXXYYYYXXXXX.best.hdf5")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 110250, 1)]       0         
_________________________________________________________________
conv1d (Conv1D)              (None, 36750, 128)        512       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 36750, 128)        49280     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 12250, 128)        0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12250, 128)        49280     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4083, 128)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4083, 256)         98560 

## Evaluate the Model on Frames

### Create Test Matrix

In [9]:
# Create a big matrix to accomodate train files
spchnumber_test = len(test_list)
X_test = np.ones( (spchnumber_test, frame_size), dtype = float )
print( X_test.shape )

# Fill the matrix with data 
i = 0
for spch1 in test_path:
    #print( len(spch1) )
    X_test[i,:] = np.asarray( sf.read( spch1 )[0] )
    # MF
    #X_test[i,0:110080] = np.asarray( sf.read( spch1 )[0] )
    # MF
    #X_test[i,110081:frame_size] = 0    
    i += 1
    if (i%10000 == 0.000):
        print("Files read: "+str(i) )

print( "Data matrix filled: "+str(i)+" files" )

#expanding shapes for Conv1D
X_test = np.expand_dims( X_test, axis = 2)

(7011, 110250)
Data matrix filled: 7011 files


In [10]:
X_test

array([[[ 0.06903076],
        [ 0.07232666],
        [ 0.03991699],
        ...,
        [-0.00299072],
        [ 0.10144043],
        [ 0.06402588]],

       [[-0.02331543],
        [-0.02386475],
        [ 0.01455688],
        ...,
        [-0.26556396],
        [-0.40948486],
        [-0.49389648]],

       [[-0.06365967],
        [-0.07427979],
        [-0.08480835],
        ...,
        [-0.00506592],
        [-0.05490112],
        [-0.10498047]],

       ...,

       [[-0.46313477],
        [-0.57794189],
        [-0.41906738],
        ...,
        [-0.10498047],
        [-0.12585449],
        [-0.05508423]],

       [[-0.14581299],
        [-0.17462158],
        [-0.19876099],
        ...,
        [ 0.11569214],
        [ 0.10952759],
        [ 0.10638428]],

       [[-0.11608887],
        [-0.17147827],
        [-0.1842041 ],
        ...,
        [-0.46838379],
        [-0.47943115],
        [-0.4647522 ]]])

In [11]:
# Create a vector for labels
# Binary class metrics

from keras.utils import np_utils, to_categorical
Y_test = to_categorical( test_label_list, num_classes = nb_classes )

In [12]:
Y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [13]:
# Evaluating the model on test set
fold = Test
print("Evaluating on: "+str(fold) ) 
print( model.evaluate( X_test, Y_test, verbose = 1) )
print( model.metrics_names )

Evaluating on: fold1
220/220 [==============================] - 159s 724ms/step - loss: 0.0834 - accuracy: 0.9772
[0.0861172005534172, 0.9743260741233826]
['loss', 'accuracy']


In [14]:
t_test = test_list
s_test = test_seg_list

In [15]:
#Combining similar segments

big_matrix = []

print("Testing on test set of fold number: " + str(fold))

tested = list()
h = int(max(s_test)) # all frames available
    
for i in range(len(X_test)):
    
    # if t_valid[i] not in tested:
    if t_test[i].split('_')[0] not in tested:    

        #tested.append(t_valid[i])
        tested.append( t_test[i].split('_')[0] )
        #similar_items_res = list()
        similar_items_res_sum = list()
        #similar_items_conf = list()

        #search for similar items
        
        count_frames = 1
        j = 0
            
        while j < len(X_test):
            #if ( t_valid[i] == t_valid[j] and s_valid[j] == count_frames):
            if ( t_test[i].split('_')[0] == t_test[j].split('_')[0] and s_test[j] == count_frames):
                if ( count_frames <= h ): 
                    #compute the results for similar items
                    #y_proba   = model.predict(X_valid[j:j+1])
                    #y_predict = y_proba.argmax()
                    #y_target  = Y_valid[j:j+1].argmax(axis=-1) 
                    
                    y_proba   = model.predict( X_test[j:j+1] )
                    y_predict = y_proba.argmax()
                    y_target  = Y_test[j:j+1].argmax( axis=-1 ) 
                    
                    
                    
                    #similar_items_res.append(y_predict) # y_predict = y_proba.argmax()
                    #similar_items_res_sum.append(y_proba)
                    similar_items_res_sum.append( y_proba ) 
                    
                    count_frames += 1
                    j = 0
            j += 1
        
        big_matrix.append([similar_items_res_sum, y_target])
        
print("Done testing - Total samples:", len(big_matrix))

Testing on test set of fold number: fold1
Done testing - Total samples: 334


In [16]:
max(s_test)

21

In [17]:
#Insert the number of splits (Example: 5 splits for 21 frames -> 1, 6, 12, 18, 21)

splits = [1, 2, 5, 10, 15, 20, 21, 23, 25]
        
percentage_vot = []
percentage_sum  = []

for i in range (len(splits)):
    percentage_vot.append(0)
    percentage_sum.append(0)
    
for obj_similiar_items_res_sum in big_matrix:

    similar_items_res_sum = obj_similiar_items_res_sum[0]
    y_target = obj_similiar_items_res_sum[1]
    
    beg = 0
    index = 0
    lens = len(similar_items_res_sum) #To access the list of similar objects
    candidates_vot = np.zeros(nb_classes)
    candidates_sum = np.ones(nb_classes)
    
    for split in splits:
        
        current_split = split
        end = min(lens, split)
            
        for n in range(beg, end):
            candidates_vot[similar_items_res_sum[n].argmax()] += 1 #voting
            candidates_sum = candidates_sum + similar_items_res_sum[n] #sum
            
        if (np.argmax(candidates_vot) == y_target):
            percentage_vot[index] += 1
        if (np.argmax(candidates_sum) == y_target):
            percentage_sum[index] += 1
            
        candidates_sum = candidates_sum.copy()
        candidates_vot = candidates_vot.copy()
            
        beg = current_split
        index += 1

In [18]:
# VOTING RULE
i = 0
for split in percentage_vot:
    samples = len(big_matrix)
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1

Acc: 97.0059880239521 Number of tracks tested: 334 Number of frames combined: 1
Acc: 96.7065868263473 Number of tracks tested: 334 Number of frames combined: 2
Acc: 98.80239520958084 Number of tracks tested: 334 Number of frames combined: 5
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 10
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 15
Acc: 99.40119760479041 Number of tracks tested: 334 Number of frames combined: 20
Acc: 99.40119760479041 Number of tracks tested: 334 Number of frames combined: 21
Acc: 99.40119760479041 Number of tracks tested: 334 Number of frames combined: 23
Acc: 99.40119760479041 Number of tracks tested: 334 Number of frames combined: 25


In [19]:
# SUM RULE
i = 0
for split in percentage_sum:
    samples = len(big_matrix)
    print( 'Acc:', ((split/samples)*100), 'Number of tracks tested:', samples, 'Number of frames combined:', splits[i])
    i += 1

Acc: 97.0059880239521 Number of tracks tested: 334 Number of frames combined: 1
Acc: 97.60479041916167 Number of tracks tested: 334 Number of frames combined: 2
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 5
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 10
Acc: 98.80239520958084 Number of tracks tested: 334 Number of frames combined: 15
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 20
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 21
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 23
Acc: 99.10179640718563 Number of tracks tested: 334 Number of frames combined: 25


In [23]:
print("Notebook Ended")

Notebook Ended


## TRAINED on STFT Original AUDIO???

In [ ]:
MF
Acc: 50.60606060606061 Number of tracks tested: 330 Number of frames combined: 1
Acc: 52.72727272727272 Number of tracks tested: 330 Number of frames combined: 2
Acc: 56.666666666666664 Number of tracks tested: 330 Number of frames combined: 5
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 10
Acc: 58.484848484848484 Number of tracks tested: 330 Number of frames combined: 15
Acc: 58.78787878787879 Number of tracks tested: 330 Number of frames combined: 20
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 21
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 23
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 25

Acc: 50.60606060606061 Number of tracks tested: 330 Number of frames combined: 1
Acc: 52.121212121212125 Number of tracks tested: 330 Number of frames combined: 2
Acc: 55.45454545454545 Number of tracks tested: 330 Number of frames combined: 5
Acc: 59.3939393939394 Number of tracks tested: 330 Number of frames combined: 10
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 15
Acc: 60.303030303030305 Number of tracks tested: 330 Number of frames combined: 20
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 21
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 23
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
MF_FGSM

Acc: 50.60606060606061 Number of tracks tested: 330 Number of frames combined: 1
Acc: 52.72727272727272 Number of tracks tested: 330 Number of frames combined: 2
Acc: 56.666666666666664 Number of tracks tested: 330 Number of frames combined: 5
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 10
Acc: 58.484848484848484 Number of tracks tested: 330 Number of frames combined: 15
Acc: 58.78787878787879 Number of tracks tested: 330 Number of frames combined: 20
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 21
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 23
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 25

            
Acc: 50.60606060606061 Number of tracks tested: 330 Number of frames combined: 1
Acc: 52.121212121212125 Number of tracks tested: 330 Number of frames combined: 2
Acc: 55.45454545454545 Number of tracks tested: 330 Number of frames combined: 5
Acc: 59.3939393939394 Number of tracks tested: 330 Number of frames combined: 10
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 15
Acc: 60.303030303030305 Number of tracks tested: 330 Number of frames combined: 20
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 21
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 23
Acc: 61.212121212121204 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
MF_BIM
Acc: 49.09090909090909 Number of tracks tested: 330 Number of frames combined: 1
Acc: 51.81818181818182 Number of tracks tested: 330 Number of frames combined: 2
Acc: 56.060606060606055 Number of tracks tested: 330 Number of frames combined: 5
Acc: 57.878787878787875 Number of tracks tested: 330 Number of frames combined: 10
Acc: 58.78787878787879 Number of tracks tested: 330 Number of frames combined: 15
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 20
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 21
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 23
Acc: 60.0 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 49.09090909090909 Number of tracks tested: 330 Number of frames combined: 1
Acc: 51.21212121212121 Number of tracks tested: 330 Number of frames combined: 2
Acc: 55.757575757575765 Number of tracks tested: 330 Number of frames combined: 5
Acc: 59.696969696969695 Number of tracks tested: 330 Number of frames combined: 10
Acc: 60.60606060606061 Number of tracks tested: 330 Number of frames combined: 15
Acc: 60.60606060606061 Number of tracks tested: 330 Number of frames combined: 20
Acc: 60.60606060606061 Number of tracks tested: 330 Number of frames combined: 21
Acc: 60.60606060606061 Number of tracks tested: 330 Number of frames combined: 23
Acc: 60.60606060606061 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
STFT

Acc: 62.727272727272734 Number of tracks tested: 330 Number of frames combined: 1
Acc: 64.24242424242425 Number of tracks tested: 330 Number of frames combined: 2
Acc: 66.66666666666666 Number of tracks tested: 330 Number of frames combined: 5
Acc: 67.57575757575758 Number of tracks tested: 330 Number of frames combined: 10
Acc: 71.21212121212122 Number of tracks tested: 330 Number of frames combined: 15
Acc: 71.81818181818181 Number of tracks tested: 330 Number of frames combined: 20
Acc: 71.81818181818181 Number of tracks tested: 330 Number of frames combined: 21
Acc: 71.81818181818181 Number of tracks tested: 330 Number of frames combined: 23
Acc: 71.81818181818181 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 62.727272727272734 Number of tracks tested: 330 Number of frames combined: 1
Acc: 65.75757575757576 Number of tracks tested: 330 Number of frames combined: 2
Acc: 67.27272727272727 Number of tracks tested: 330 Number of frames combined: 5
Acc: 68.78787878787878 Number of tracks tested: 330 Number of frames combined: 10
Acc: 71.81818181818181 Number of tracks tested: 330 Number of frames combined: 15
Acc: 71.51515151515152 Number of tracks tested: 330 Number of frames combined: 20
Acc: 71.51515151515152 Number of tracks tested: 330 Number of frames combined: 21
Acc: 71.51515151515152 Number of tracks tested: 330 Number of frames combined: 23
Acc: 71.51515151515152 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
STFT_BIM

Acc: 26.36363636363636 Number of tracks tested: 330 Number of frames combined: 1
Acc: 27.27272727272727 Number of tracks tested: 330 Number of frames combined: 2
Acc: 26.060606060606062 Number of tracks tested: 330 Number of frames combined: 5
Acc: 26.666666666666668 Number of tracks tested: 330 Number of frames combined: 10
Acc: 27.575757575757574 Number of tracks tested: 330 Number of frames combined: 15
Acc: 26.969696969696972 Number of tracks tested: 330 Number of frames combined: 20
Acc: 26.969696969696972 Number of tracks tested: 330 Number of frames combined: 21
Acc: 26.969696969696972 Number of tracks tested: 330 Number of frames combined: 23
Acc: 26.969696969696972 Number of tracks tested: 330 Number of frames combined: 25

Acc: 26.36363636363636 Number of tracks tested: 330 Number of frames combined: 1
Acc: 28.18181818181818 Number of tracks tested: 330 Number of frames combined: 2
Acc: 26.36363636363636 Number of tracks tested: 330 Number of frames combined: 5
Acc: 27.27272727272727 Number of tracks tested: 330 Number of frames combined: 10
Acc: 28.78787878787879 Number of tracks tested: 330 Number of frames combined: 15
Acc: 27.575757575757574 Number of tracks tested: 330 Number of frames combined: 20
Acc: 27.575757575757574 Number of tracks tested: 330 Number of frames combined: 21
Acc: 27.575757575757574 Number of tracks tested: 330 Number of frames combined: 23
Acc: 27.575757575757574 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
STFT_FGSM

Acc: 21.818181818181817 Number of tracks tested: 330 Number of frames combined: 1
Acc: 22.424242424242426 Number of tracks tested: 330 Number of frames combined: 2
Acc: 23.939393939393938 Number of tracks tested: 330 Number of frames combined: 5
Acc: 23.333333333333332 Number of tracks tested: 330 Number of frames combined: 10
Acc: 25.151515151515152 Number of tracks tested: 330 Number of frames combined: 15
Acc: 23.333333333333332 Number of tracks tested: 330 Number of frames combined: 20
Acc: 23.03030303030303 Number of tracks tested: 330 Number of frames combined: 21
Acc: 23.03030303030303 Number of tracks tested: 330 Number of frames combined: 23
Acc: 23.03030303030303 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 21.818181818181817 Number of tracks tested: 330 Number of frames combined: 1
Acc: 22.424242424242426 Number of tracks tested: 330 Number of frames combined: 2
Acc: 24.242424242424242 Number of tracks tested: 330 Number of frames combined: 5
Acc: 23.636363636363637 Number of tracks tested: 330 Number of frames combined: 10
Acc: 23.939393939393938 Number of tracks tested: 330 Number of frames combined: 15
Acc: 23.939393939393938 Number of tracks tested: 330 Number of frames combined: 20
Acc: 23.333333333333332 Number of tracks tested: 330 Number of frames combined: 21
Acc: 23.333333333333332 Number of tracks tested: 330 Number of frames combined: 23
Acc: 23.333333333333332 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
stft_griffin

Acc: 5.151515151515151 Number of tracks tested: 330 Number of frames combined: 1
Acc: 4.242424242424243 Number of tracks tested: 330 Number of frames combined: 2
Acc: 4.545454545454546 Number of tracks tested: 330 Number of frames combined: 5
Acc: 3.0303030303030303 Number of tracks tested: 330 Number of frames combined: 10
Acc: 3.0303030303030303 Number of tracks tested: 330 Number of frames combined: 15
Acc: 3.0303030303030303 Number of tracks tested: 330 Number of frames combined: 20
Acc: 1.2121212121212122 Number of tracks tested: 330 Number of frames combined: 21
Acc: 1.2121212121212122 Number of tracks tested: 330 Number of frames combined: 23
Acc: 1.2121212121212122 Number of tracks tested: 330 Number of frames combined: 25
    
Acc: 5.151515151515151 Number of tracks tested: 330 Number of frames combined: 1
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 2
Acc: 2.1212121212121215 Number of tracks tested: 330 Number of frames combined: 5
Acc: 2.1212121212121215 Number of tracks tested: 330 Number of frames combined: 10
Acc: 1.5151515151515151 Number of tracks tested: 330 Number of frames combined: 15
Acc: 0.6060606060606061 Number of tracks tested: 330 Number of frames combined: 20
Acc: 0.9090909090909091 Number of tracks tested: 330 Number of frames combined: 21
Acc: 0.9090909090909091 Number of tracks tested: 330 Number of frames combined: 23
Acc: 0.9090909090909091 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
stft_griffin_bim

Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 1
Acc: 8.181818181818182 Number of tracks tested: 330 Number of frames combined: 2
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 5
Acc: 14.84848484848485 Number of tracks tested: 330 Number of frames combined: 10
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 15
Acc: 17.87878787878788 Number of tracks tested: 330 Number of frames combined: 20
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 21
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 23
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 1
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 5
Acc: 16.969696969696972 Number of tracks tested: 330 Number of frames combined: 10
Acc: 16.363636363636363 Number of tracks tested: 330 Number of frames combined: 15
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 20
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 21
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 23
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 25

In [ ]:
stft_griffin_fgsm

Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 1
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 2
Acc: 14.545454545454545 Number of tracks tested: 330 Number of frames combined: 5
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 10
Acc: 17.87878787878788 Number of tracks tested: 330 Number of frames combined: 15
Acc: 18.787878787878785 Number of tracks tested: 330 Number of frames combined: 20
Acc: 18.484848484848484 Number of tracks tested: 330 Number of frames combined: 21
Acc: 18.484848484848484 Number of tracks tested: 330 Number of frames combined: 23
Acc: 18.484848484848484 Number of tracks tested: 330 Number of frames combined: 25
        
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 1
Acc: 14.242424242424242 Number of tracks tested: 330 Number of frames combined: 2
Acc: 16.363636363636363 Number of tracks tested: 330 Number of frames combined: 5
Acc: 16.363636363636363 Number of tracks tested: 330 Number of frames combined: 10
Acc: 17.575757575757574 Number of tracks tested: 330 Number of frames combined: 15
Acc: 18.484848484848484 Number of tracks tested: 330 Number of frames combined: 20
Acc: 18.181818181818183 Number of tracks tested: 330 Number of frames combined: 21
Acc: 18.181818181818183 Number of tracks tested: 330 Number of frames combined: 23
Acc: 18.181818181818183 Number of tracks tested: 330 Number of frames combined: 25

## TRAINED on STFT Reconstructed Griffin-Lin

In [ ]:
STFT_Griffin
Acc: 35.15151515151515 Number of tracks tested: 330 Number of frames combined: 1
Acc: 34.84848484848485 Number of tracks tested: 330 Number of frames combined: 2
Acc: 48.484848484848484 Number of tracks tested: 330 Number of frames combined: 5
Acc: 50.0 Number of tracks tested: 330 Number of frames combined: 10
Acc: 53.63636363636364 Number of tracks tested: 330 Number of frames combined: 15
Acc: 56.96969696969697 Number of tracks tested: 330 Number of frames combined: 20
Acc: 56.96969696969697 Number of tracks tested: 330 Number of frames combined: 21
Acc: 56.96969696969697 Number of tracks tested: 330 Number of frames combined: 23
Acc: 56.96969696969697 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 35.15151515151515 Number of tracks tested: 330 Number of frames combined: 1
Acc: 40.909090909090914 Number of tracks tested: 330 Number of frames combined: 2
Acc: 51.21212121212121 Number of tracks tested: 330 Number of frames combined: 5
Acc: 55.15151515151515 Number of tracks tested: 330 Number of frames combined: 10
Acc: 55.757575757575765 Number of tracks tested: 330 Number of frames combined: 15
Acc: 57.57575757575758 Number of tracks tested: 330 Number of frames combined: 20
Acc: 59.3939393939394 Number of tracks tested: 330 Number of frames combined: 21
Acc: 59.3939393939394 Number of tracks tested: 330 Number of frames combined: 23
Acc: 59.3939393939394 Number of tracks tested: 330 Number of frames combined: 25
            
STFT_Griffin_FGSM
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 1
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 2
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 5
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 10
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 15
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 20
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 21
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 23
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 25

Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 1
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 2
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 5
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 10
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 15
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 20
Acc: 10.0 Number of tracks tested: 330 Number of frames combined: 21
Acc: 10.0 Number of tracks tested: 330 Number of frames combined: 23
Acc: 10.0 Number of tracks tested: 330 Number of frames combined: 25
            
STFT_Griffin_Bim
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 1
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 5
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 10
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 15
Acc: 10.909090909090908 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 25

Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 1
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 5
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 10
Acc: 10.303030303030303 Number of tracks tested: 330 Number of frames combined: 15
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 20
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 21
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 23
Acc: 10.606060606060606 Number of tracks tested: 330 Number of frames combined: 25
            
STFT
Acc: 15.454545454545453 Number of tracks tested: 330 Number of frames combined: 1
Acc: 17.87878787878788 Number of tracks tested: 330 Number of frames combined: 2
Acc: 14.84848484848485 Number of tracks tested: 330 Number of frames combined: 5
Acc: 17.87878787878788 Number of tracks tested: 330 Number of frames combined: 10
Acc: 18.787878787878785 Number of tracks tested: 330 Number of frames combined: 15
Acc: 16.969696969696972 Number of tracks tested: 330 Number of frames combined: 20
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 21
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 23
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 25

Acc: 15.454545454545453 Number of tracks tested: 330 Number of frames combined: 1
Acc: 17.272727272727273 Number of tracks tested: 330 Number of frames combined: 2
Acc: 15.151515151515152 Number of tracks tested: 330 Number of frames combined: 5
Acc: 17.87878787878788 Number of tracks tested: 330 Number of frames combined: 10
Acc: 18.181818181818183 Number of tracks tested: 330 Number of frames combined: 15
Acc: 16.363636363636363 Number of tracks tested: 330 Number of frames combined: 20
Acc: 16.969696969696972 Number of tracks tested: 330 Number of frames combined: 21
Acc: 16.969696969696972 Number of tracks tested: 330 Number of frames combined: 23
Acc: 16.969696969696972 Number of tracks tested: 330 Number of frames combined: 25
            
STFT_FGSM
Acc: 5.454545454545454 Number of tracks tested: 330 Number of frames combined: 1
Acc: 5.151515151515151 Number of tracks tested: 330 Number of frames combined: 2
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 5
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 10
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 15
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 20
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 21
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 23
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 5.454545454545454 Number of tracks tested: 330 Number of frames combined: 1
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 2
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 5
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 10
Acc: 6.0606060606060606 Number of tracks tested: 330 Number of frames combined: 15
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 20
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 21
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 23
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 25
            
STFT_BIM
Acc: 7.878787878787878 Number of tracks tested: 330 Number of frames combined: 1
Acc: 6.969696969696971 Number of tracks tested: 330 Number of frames combined: 2
Acc: 6.363636363636363 Number of tracks tested: 330 Number of frames combined: 5
Acc: 8.181818181818182 Number of tracks tested: 330 Number of frames combined: 10
Acc: 7.575757575757576 Number of tracks tested: 330 Number of frames combined: 15
Acc: 7.575757575757576 Number of tracks tested: 330 Number of frames combined: 20
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 21
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 23
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 7.878787878787878 Number of tracks tested: 330 Number of frames combined: 1
Acc: 7.878787878787878 Number of tracks tested: 330 Number of frames combined: 2
Acc: 6.666666666666667 Number of tracks tested: 330 Number of frames combined: 5
Acc: 8.181818181818182 Number of tracks tested: 330 Number of frames combined: 10
Acc: 8.181818181818182 Number of tracks tested: 330 Number of frames combined: 15
Acc: 7.575757575757576 Number of tracks tested: 330 Number of frames combined: 20
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 21
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 23
Acc: 7.2727272727272725 Number of tracks tested: 330 Number of frames combined: 25
            
MF
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 1
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 2
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 5
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 10
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 15
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 25
            
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 1
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 2
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 5
Acc: 13.939393939393941 Number of tracks tested: 330 Number of frames combined: 10
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 15
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 20
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 21
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 23
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 25
            
MF_BIM
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 1
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 5
Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 10
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 15
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 25

Acc: 12.424242424242424 Number of tracks tested: 330 Number of frames combined: 1
Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 5
Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 10
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 15
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 25

MF_FGSM
Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 1
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 5
Acc: 12.727272727272727 Number of tracks tested: 330 Number of frames combined: 10
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 15
Acc: 11.212121212121213 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.515151515151516 Number of tracks tested: 330 Number of frames combined: 25

Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 1
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 2
Acc: 13.636363636363635 Number of tracks tested: 330 Number of frames combined: 5
Acc: 13.333333333333334 Number of tracks tested: 330 Number of frames combined: 10
Acc: 13.030303030303031 Number of tracks tested: 330 Number of frames combined: 15
Acc: 12.121212121212121 Number of tracks tested: 330 Number of frames combined: 20
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 21
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 23
Acc: 11.818181818181818 Number of tracks tested: 330 Number of frames combined: 25